# Generating Python Code with Granite

**NOTE:** This recipe demonstrates how to use Granite Models to generate Python code from text prompts.

## Setting Up

### Install dependencies

In [ ]:
%pip install "git+https://github.com/ibm-granite-community/utils" \
    langchain_community \
    transformers \
    replicate \
    pandas

## Select and import the model from Replicate

In [ ]:
from langchain_community.llms import Replicate
from transformers import AutoTokenizer
from ibm_granite_community.notebook_utils import get_env_var
from ibm_granite_community.langchain import TokenizerChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from textwrap import dedent

# Model configuration
model_path = "ibm-granite/granite-3.3-8b-instruct"

# Initialize Replicate model
model = Replicate(
    model=model_path,
    replicate_api_token=get_env_var('REPLICATE_API_TOKEN'),
)

## Configure the model to use tokenizer and chat template with a system message

In [ ]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Create a system message for the chat prompt
system_prompt = SystemMessage(content=dedent("""\
    You are a Python code generation specialist. Your task is to perform Fill-in-the-Middle (FIM) code completion following these strict guidelines:

    1. Generate ONLY executable Python code without any explanations or comments outside of docstrings
    2. Follow PEP 8 style guidelines and best practices
    3. Include appropriate type hints for all function parameters and return values
    4. Write comprehensive docstrings following Google docstring format
    5. Ensure the generated code fits seamlessly between the provided prefix and suffix
    6. Maintain consistent indentation and naming conventions with surrounding code
    7. Optimize for readability, efficiency, and maintainability
    8. Do not include any natural language responses or explanations outside of code

    Your response must consist solely of valid Python code that can be directly executed.
"""))

# Create a tokenizer chat prompt template with the system message
chat_template_with_system_message = TokenizerChatPromptTemplate.from_messages(
    [
        system_prompt,
        ("human", "{question}"),
    ],
    tokenizer=tokenizer
)

chain_sys = chat_template_with_system_message | model

## Configure the model to use the FIM (Fill-in-the-Middle) format

In [ ]:
def fim_pipeline(prefix, suffix=""):
    # Format FIM input with special tokens
    input_text = f'<fim_prefix>{prefix}<fim_suffix>{suffix}<fim_middle>'

    # Generate using the Replicate model chain with system prompt
    response = chain_sys.invoke({"question": input_text})

    return response.strip()

### What is Fill-in-the-Middle (FIM)?
Fill-in-the-Middle (FIM) is a technique where the model is provided with the beginning and end of a code snippet and is asked to complete the missing logic in the middle. This method is highly effective for:
1) Completing core programming constructs like functions, methods, conditionals, and loops
2) Implementing common algorithms (e.g., prime checks, factorials, string reversal)
3) Demonstrating language-agnostic problem-solving across languages like Python, C/C++, Go, and Java
4) Teaching introductory programming concepts by auto-completing starter code
5) Assisting learners with template-based coding exercises
6) Rapidly generating idiomatic solutions using best practices in each language

### Example №1: Bug Fixing (missing loop logic)

In [ ]:
prefix = """
def sum_even_numbers(lst):
    total = 0
"""
suffix = """
    return total
"""

print(fim_pipeline(prefix=prefix, suffix=suffix))

In [ ]:
# Paste this code into a Python environment to run it

def sum_even_numbers(lst):
    total = 0
    for num in lst:
        if num % 2 == 0:
            total += num
    return total

### Example №2: Adding Missing Implementation (class method logic)


In [ ]:
prefix = """
class Counter:
    def __init__(self):
        self.count = 0

    def increment(self):
"""
suffix = """
    return self.count
"""

print(fim_pipeline(prefix=prefix, suffix=suffix))

In [ ]:
# Paste this code into a Python environment to run it

class Counter:
    def __init__(self):
        self.count = 0

    def increment(self):
        self.count += 1
        return self.count

    def decrement(self):
        self.count -= 1
        return self.count

    def reset(self):
        self.count = 0
        return self.count

### Example №3: Loop Completion for Filtering


In [ ]:
prefix = """
def filter_positive(nums):
    result = []
"""
suffix = """
    return result
"""

print(fim_pipeline(prefix=prefix, suffix=suffix))

In [ ]:
# Paste this code into a Python environment to run it

def filter_positive(nums):
    result = []
    for num in nums:
        if num > 0:
            result.append(num)
    return result

### Example №4: Error Handling Completion


In [ ]:
prefix = """
def read_file(path):
    try:
        with open(path, 'r') as file:
"""
suffix = """
    except FileNotFoundError:
        return "File not found."
"""

print(fim_pipeline(prefix=prefix, suffix=suffix))

In [ ]:
# Paste this code into a Python environment to run it

def read_file(path):
    try:
        with open(path, 'r') as file:
            content = file.read()
            return content
    except FileNotFoundError:
        return "File not found."

### Example №5: Data Transformation with List Comprehension


In [ ]:
prefix = """
def square_even(nums):
"""
suffix = """
    return result
"""

print(fim_pipeline(prefix=prefix, suffix=suffix, ))

In [ ]:
# Paste this code into a Python environment to run it

def square_even(nums):
    result = [num ** 2 for num in nums if num % 2 == 0]
    return result

### Example №6: Partial Code Completion in Data Analysis Context


In [ ]:
prefix = """
import pandas as pd

def load_and_clean(filepath):
    df = pd.read_csv(filepath)
"""
suffix = """
    return df
"""

print(fim_pipeline(prefix=prefix, suffix=suffix))

In [ ]:
# Paste this code into a Python environment to run it

import pandas as pd

def load_and_clean(filepath):
    df = pd.read_csv(filepath)
    df.dropna(inplace=True)  # Remove missing values
    df.reset_index(drop=True, inplace=True)  # Reset index after dropping rows
    df['column_name'] = df['column_name'].astype('int')
    return df